## DJI Tello Drone - YOLOv5


In [1]:
import matplotlib.pyplot as plt
import cv2
import torch
import threading

In [ ]:
#Ref. https://github.com/ultralytics/yolov5
#NOTE: Connect to internet, after this cell is executed, connect to drone

path = "D:/Python/UPH_Internship_2023/Ball_Track/NewBallDataset.pt"
model = torch.hub.load("ultralytics/yolov5", "custom", path, force_reload=True)

In [ ]:
# Thanks Github Copilot
def detect_yolo(video):
    """Detect from video file then cv2.imshow"""
    cap = cv2.VideoCapture(video)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            results = model(frame)
            results.render() #NOTE YOLO render function replaces old image with annotated image
            cv2.imshow("frame", frame) #NOTE call imshow with old variable
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

            """get names of detected objects then print name confidence and bounding box dimensions"""
            for i in range(len(results.xyxy[0])):
                name = results.names[int(results.xyxy[0][i][5])]
                conf = results.xyxy[0][i][4]
                print(name, conf)
                x1 = int(results.xyxy[0][i][0])
                y1 = int(results.xyxy[0][i][1])
                x2 = int(results.xyxy[0][i][2])
                y2 = int(results.xyxy[0][i][3])
                print(x1, y1, x2, y2)
            
        else:
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:

#detect_yolo(0)

In [ ]:
#NOTE Run this cell if window gets stuck
try:
    cv2.destroyAllWindows()
except:
    print("No cv2 windows open")

In [ ]:
from djitellopy import Tello
from time import sleep
#Ref. https://github.com/damiafuentes/DJITelloPy/blob/master/examples/take-picture.py
#NOTE when this runs make sure you are connected to the drone wifi

tello = Tello()
tello.connect()

tello.streamon()
sleep(2)

#BUG window stuck so a a force quit is needed. Please restart the Python kernel tqo continue
#NOTE this function loads slowly

def yolo_drone(tello):
    """get video from tello then detect"""
    run = True
    """make a thread to keep the drone alive"""
    t = threading.Thread(target=keep_alive_drone, args=(tello,))
    t.start()
    t_running = True
    print("Thread Start")
    i = 0
    while run == True:        
        try:
                
            #this loading part is  s l o w  at the start
            #BUG drone timeouts b4 loading is done
            my_frame = tello.get_frame_read().frame
            results = model(my_frame)
            results.render()
            cv2.imshow("frame", my_frame)
                       
            if cv2.waitKey(1) & 0xFF == ord("q"):
                run=False
                cv2.destroyAllWindows()
                return
            """get tello camera resolution"""
            width = tello.get_video_capture().get(3)
            height = tello.get_video_capture().get(4)
            #print(width, height)

            """set bounding box dimensions"""
            x1_min = 0.15*width
            x1_max = width
            y1_min = 0.15*height
            y1_max = height
        
            """get names of detected objects then print name confidence and bounding box dimensions"""
            for i in range(len(results.xyxy[0])):
                try:
                    name = results.names[int(results.xyxy[0][i][5])]
                    if name!="balls": continue
                    
                    conf = results.xyxy[0][i][4]
                    print(name, conf)
                    x1 = int(results.xyxy[0][i][0]) #x1 is left side of bounding box
                    y1 = int(results.xyxy[0][i][1]) #y1 is top side of bounding box
                    x2 = int(results.xyxy[0][i][2]) #x2 is right side of bounding box
                    y2 = int(results.xyxy[0][i][3]) #y2 is bottom side of bounding box

                    """move drone if object is out of bounds L/R"""
                    if x1<x1_min:
                        tello.move_left(20)
                        print("move left")
                    if x1>x1_max-x1_min: #keep centered
                        tello.move_right(20)
                        print("move right")
                    """move drone if object is out of bounds U/D"""
                    if y1>y1_min:
                        tello.move_down(20)
                        print("move down")
                    if y1<y1_max-y1_min: #keep centered
                        tello.move_up(20)
                        print("move up")
                    print(x1, y1, x2, y2)
                except:
                    pass

        except:
            pass

def keep_alive_drone(tello):
    sleep(1)
    while True:
        tello.rotate_clockwise(10)
        sleep(3)
        tello.rotate_counter_clockwise(10)
        sleep(3)


In [ ]:
print("Battery: ",tello.get_battery())
def confirm(input):
    """if input is not y then stop jupyter"""
    if input.lower() != "y":
        raise SystemExit
confirm(input=input("Continue Y/N"))
print("ok")

try:
    tello.takeoff()
except:
    pass

            
sleep(2)
#t2.start()
yolo_drone(tello)

In [ ]:
tello.get_battery()

In [ ]:
tello.streamoff()

In [ ]:
# Next Step - Move drone to the center of the detected object
tello.land()